# TensorFlow 变量命名管理机制（一）

初学者经常会遇到类似于下面的错误，特别是在使用 notebook 写代码的时候：

 <font color='red'>ValueError: </font>Variable XXX... already exists, disallowed.<br/>
 
现在我们来探讨一下 TensorFlow 的命名管理机制，相信看完后你就能够很好地解决这个问题了。

In [1]:
import warnings
warnings.filterwarnings('ignore')  # 不打印 warning 

import tensorflow as tf

# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

## 1.使用 tf.Variable() 和 tf.get_variable() 创建变量

### 1.1 使用 tf.Variable() 创建变量，如果 name 一样的话会自动处理。

In [2]:
v1 = tf.Variable(initial_value=[1.0], name='v')
v2 = tf.Variable(initial_value=[2.0], name='v')
v3 = tf.Variable(initial_value=[1.0, 2.0], name='v')
print('v1', v1)
print('v2', v2)
print('v3', v3)

v1 <tf.Variable 'v:0' shape=(1,) dtype=float32_ref>
v2 <tf.Variable 'v_1:0' shape=(1,) dtype=float32_ref>
v3 <tf.Variable 'v_2:0' shape=(2,) dtype=float32_ref>


### 1.2 使用 tf.get_variable() 创建变量，不能用一样的 name 

In [3]:
gv1 = tf.get_variable(name='gv', shape=[2,3], initializer=tf.truncated_normal_initializer())
# gv2 = tf.get_variable(name='gv', shape=[2,3], initializer=tf.truncated_normal_initializer())   # ValueError: Variable gv already exists
print('gv1', gv1)

gv1 <tf.Variable 'gv:0' shape=(2, 3) dtype=float32_ref>


如果非要使用一样的 name 的话，那么需要结合 tf.variable_scope 并设置参数 reuse=True，后面会介绍。

### 1.3 tf.Variable() 和 tf.get_variable() 同时创建变量，会自动处理。

In [4]:
var1 = tf.Variable(initial_value=[1.0], name='var', trainable=False)
var2 = tf.get_variable(name='var', shape=[2,3])
var3 = tf.Variable(initial_value=[1.0], name='var')
# var4 = tf.get_variable(name='var', shape=[2,3])  # 报错
print('var1', var1)
print('var2', var2)
print('var3', var3)

var1 <tf.Variable 'var:0' shape=(1,) dtype=float32_ref>
var2 <tf.Variable 'var_1:0' shape=(2, 3) dtype=float32_ref>
var3 <tf.Variable 'var_2:0' shape=(1,) dtype=float32_ref>


注意上面，如果 tf.get_variable() 的名字和 tf.Variable() 的名字冲突的话也是会自动处理的。

### 1.4 使用 tf.placeholder() 创建占位符
tf.placeholder 创建的并不是变量，这里也提一下只是想看看它的命名是什么样子的。

In [5]:
ph1 = tf.placeholder(dtype=tf.float32, shape=[1,3], name='ph')
ph2 = tf.placeholder(dtype=tf.float32, shape=[2,3], name='ph')

print('ph1:', ph1)
print('ph2:', ph2)

ph1: Tensor("ph:0", shape=(1, 3), dtype=float32)
ph2: Tensor("ph_1:0", shape=(2, 3), dtype=float32)


可以看到我们用 tf.placeholder 挖了两个坑，坑名都是 'ph'，然后 TensorFlow 自动处理了。

### 1.5 获取全部的变量和 trainable 变量

In [6]:
all_vars = tf.global_variables()
for i in range(len(all_vars)):
    print(i, all_vars[i])

0 <tf.Variable 'v:0' shape=(1,) dtype=float32_ref>
1 <tf.Variable 'v_1:0' shape=(1,) dtype=float32_ref>
2 <tf.Variable 'v_2:0' shape=(2,) dtype=float32_ref>
3 <tf.Variable 'gv:0' shape=(2, 3) dtype=float32_ref>
4 <tf.Variable 'var:0' shape=(1,) dtype=float32_ref>
5 <tf.Variable 'var_1:0' shape=(2, 3) dtype=float32_ref>
6 <tf.Variable 'var_2:0' shape=(1,) dtype=float32_ref>


In [7]:
all_trainable_vars = tf.trainable_variables()
for i in range(len(all_trainable_vars)):
    print(i, all_trainable_vars[i])

0 <tf.Variable 'v:0' shape=(1,) dtype=float32_ref>
1 <tf.Variable 'v_1:0' shape=(1,) dtype=float32_ref>
2 <tf.Variable 'v_2:0' shape=(2,) dtype=float32_ref>
3 <tf.Variable 'gv:0' shape=(2, 3) dtype=float32_ref>
4 <tf.Variable 'var_1:0' shape=(2, 3) dtype=float32_ref>
5 <tf.Variable 'var_2:0' shape=(1,) dtype=float32_ref>


tf.global_variables() 和 tf.trainable_variables() 用来获取**所有的变量**和**所有可训的变量**。

这个在后期的时候会用到，比如：我们定义两个不同的优化器，它们各自优化部分参数，这时候我们需要给它们传入各自需要优化的变量的list，这时候它们就只优化list中的变量，其他变量它就不算梯度了。

注意上面 trainable_variables() 中少了一个 'var:0' 变量，这是因为我们把它设置成了 trainable=False, 那么在优化的过程中就不会对它求梯度了。

如果现在不太理解这个也没有关系，后面的学习中要用到这个的时候就能明白了。

## 2. 使用 tf.name_scope() 和 tf.variable_scope() 管理命名空间

为了避免和上面的名称搞混了，这里建议 **Restart Kernel 然后再继续试验。**

In [1]:
import warnings
warnings.filterwarnings('ignore')  # 不打印 warning 

import tensorflow as tf

# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

### 2.1 使用 tf.name_scope()

In [2]:
with tf.name_scope('ns1') as ns:
    ns_v1 = tf.Variable(initial_value=[1.0], name='v')
    ns_gv1 = tf.get_variable(name='v', shape=[2,3])
    ns_v2 = tf.Variable(initial_value=[1.0], name='v')
    
print('ns_v1', ns_v1)
print('ns_gv1', ns_gv1)
print('ns_v2', ns_v2)

ns_v1 <tf.Variable 'ns1/v:0' shape=(1,) dtype=float32_ref>
ns_gv1 <tf.Variable 'v:0' shape=(2, 3) dtype=float32_ref>
ns_v2 <tf.Variable 'ns1/v_1:0' shape=(1,) dtype=float32_ref>


结果就很清晰了，在 tf.name_scope() 里边：
- 使用 tf.Variable() 创建的变量，会自动地在 name 前面加上 scope name
- **使用 tf.get_variable() 创建的变量，并不受 name_scope 的影响**

### 2.2 使用 tf.variable_scope()

In [3]:
with tf.variable_scope('vs1') as vs:
    vs_v1 = tf.Variable(initial_value=[1.0], name='v')
    vs_gv1 = tf.get_variable(name='v', shape=[2,3])
    # vs_gv2 = tf.get_variable(name='v', shape=[1,3])  # ValueError: Variable vs1/v already exists

print('vs_v1', vs_v1)
print('vs_gv1', vs_gv1)

vs_v1 <tf.Variable 'vs1/v:0' shape=(1,) dtype=float32_ref>
vs_gv1 <tf.Variable 'vs1/v_1:0' shape=(2, 3) dtype=float32_ref>


从上面的结果可以看到 tf.variable_scope 和 tf.name_scope 的区别了吧：
- 使用 tf.Variable() 创建的变量，会自动地在 name 前面加上 variable scope name
- 使用 tf.get_variable() 创建的变量，也会自动地在 name 前面加上 variable scope name

### 2.3 tf.variable_scope() 中设置 reuse=True
在开始之前，先来看看 python 中怎么判断两个变量指向的对象是不是一样的。我们直接使用 **is** 关键字来判断就行了，注意不能使用 == 来判断，和 java, C++ 中不同，python 中的 == 只是判断左右两边的内容是否一致，而不是判断左右两边是不是同一个对象。

In [4]:
a = [1,2,3]
b = [1,2,3]
c = a
print('a is b: ', a is b)
print('a is c: ', a is c)

c[0] = 555  # 修改 c，a也会变化：因为 a 和 c 指向同一个对象
b[1] = 666  # 修改 b, a不受影响：因为 a 和 b 指向两个不同的对象
print('a:', a)
print('b:', b)
print('c:', c)

a is b:  False
a is c:  True
a: [555, 2, 3]
b: [1, 666, 3]
c: [555, 2, 3]


上面的结果显示，a 和 c 是指向内存中的同一个对象的，**如果修改c的内容，a 也会一起变**。而 b 虽然和 a 的内容一模一样，但是他们并不是一个对象，如果修改 b 的内容，并不会影响到 a。

**在 TensorFlow 中，我们是通过变量的 name 来区别不同的对象。** tf.variable_scope() 中的 reuse 就是是我们为了实现变量的共享。

在上面，我们已经创建了一个 variable_scope 它的 name='vs1'；在 'vs1' 里边，我们创建了一个变量：name='v'.

In [5]:
with tf.variable_scope('vs1') as vs:
    vs_gv2 = tf.get_variable(name='v2', shape=[2,3]) 
    # vs_gv3 = tf.get_variable(name='v', shape=[2,3])  # ValueError: Variable vs1/v already exists  
    
print('vs_gv2', vs_gv2)

vs_gv2 <tf.Variable 'vs1/v2:0' shape=(2, 3) dtype=float32_ref>


In [6]:
with tf.variable_scope('vs1', reuse=True) as vs:
    vs_gv3 = tf.get_variable(name='v', shape=[2,3])  
    
print('vs_gv3', vs_gv3)
print('vs_gv3 is vs_gv1: ', vs_gv3 is vs_gv1)

vs_gv3 <tf.Variable 'vs1/v_1:0' shape=(2, 3) dtype=float32_ref>
vs_gv3 is vs_gv1:  True


上面发现了没有，当我们要**在同一个 variable_scope 下面‘定义’一个已经存在的变量 name='v' 的时候，需要设置 reuse=True**，实际上两次‘定义’指向同一个内存对象。

## 3.总结

上面主要介绍了 tf.Variable() 和 tf.get_variable() 两种创建变量的方式。实际上，TensorFlow 更加鼓励使用 t.get_variable() 来创建对象，只是它稍微麻烦一丢丢。此外，我们还探索了 tf.name_scope() 和 tf.variable_scope() 两种命名空间管理。一般来说：
- 方式一：tf.name_scope() 常常和 tf.Variable() 一起来进行变量名称管理。
- 方式二：tf.variable_scope() 常常和 tf.get_variable() 一起来进行变量名称管理和**实现权值变量共享。**

在以后的学习中，我们应该尽量用方式二来创建变量。

## 4.实用示例
下面我们将通过两个简单的例子来感受一下 TensorFlow 的变量命名管理。

为了避免前面和前面的变量搞混，建议先**restart kernel**.

In [1]:
import warnings
warnings.filterwarnings('ignore')  # 不打印 warning 

import tensorflow as tf

# 设置GPU按需增长
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [2]:
def fc(X, out_size):
    """全连接层。
    Args:
        X: 2-D tensor, [batch_size, in_size]
        out_size: the size of output tensor.
    Returns:
        h_fc: 2-D tensor, [batch_size, out_size].
    """
    in_size = X.shape[1]  # 特征维度
    W = tf.get_variable('weight', shape=[in_size, out_size], initializer=tf.truncated_normal_initializer())
    b = tf.get_variable('bias', [out_size], initializer=tf.zeros_initializer())
    h_fc = tf.nn.relu(tf.nn.xw_plus_b(X, W, b), name='relu')
    return h_fc

### 4.1 先来看一个正确的例子

In [3]:
batch_size = 128  
feature_size = 50  # 特征维度
fc1_size = 64      # 第一个全连接层的神经元个数
fc2_size = 32      # 第二个全连接层的神经元个数
X_input = tf.placeholder(dtype=tf.float32, shape=[batch_size, feature_size], name='X_input')

with tf.variable_scope('fc1') as vs:
    h_fc1 = fc(X_input, out_size=fc1_size)
with tf.variable_scope('fc2') as vs:
    h_fc2 = fc(h_fc1, out_size=fc2_size)

all_vars = tf.global_variables()
for i in range(len(all_vars)):
    print('var {}: '.format(i), all_vars[i])

var 0:  <tf.Variable 'fc1/weight:0' shape=(50, 64) dtype=float32_ref>
var 1:  <tf.Variable 'fc1/bias:0' shape=(64,) dtype=float32_ref>
var 2:  <tf.Variable 'fc2/weight:0' shape=(64, 32) dtype=float32_ref>
var 3:  <tf.Variable 'fc2/bias:0' shape=(32,) dtype=float32_ref>


### 4.2 再看看一个错误的例子

In [4]:
batch_size = 128  
feature_size = 50  # 特征维度
fc1_size = 64      # 第一个全连接层的神经元个数
fc2_size = 32      # 第二个全连接层的神经元个数
X_input = tf.placeholder(dtype=tf.float32, shape=[batch_size, feature_size], name='X_input')

h_fc1 = fc(X_input, out_size=fc1_size)
h_fc2 = fc(h_fc1, out_size=fc2_size)

all_vars = tf.global_variables()
for i in range(len(all_vars)):
    print('var {}: '.format(i), all_vars[i])

ValueError: Variable weight already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-2-2a6c9c5d485a>", line 10, in fc
    W = tf.get_variable('weight', shape=[in_size, out_size], initializer=tf.truncated_normal_initializer())
  File "<ipython-input-4-368830f9fb3e>", line 7, in <module>
    h_fc1 = fc(X_input, out_size=fc1_size)
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)


如果你已经理解了 tf.variable_scope() 和 tf.get_variable() 的用法的话，相信你一下子就能明白为什么上面会出错了。在神经网络的结构定义的时候，我们应该养成比较好的习惯，在定义每个层的时候都放在一个 tf.variable_scope() 里边。

在定义整个模型的时候，也应该把整个模型放在一个 tf.variable_scope('your_model_name') 里边，这样做有个好处就是后期你想要微调某些层或者把多个小模型联合成一个大模型的时候都可以很轻松地实现。

如果你现在还不能很好地理解这里讲的内容的话也不用担心，再后面的学习中慢慢理解就行了。